# [참고문서](https://appia.tistory.com/428)

# 일자별 시가총액

- 예시

In [1]:
from pykrx import stock

df1 = stock.get_market_cap_by_date("20000101", "20201030", "000020", "y")
df1

,시가총액,거래량,거래대금,상장주식수
날짜,,,,
2000-12-31,34493892480,59001040,758103262000,5423568
2001-12-31,51338041860,57464100,538344129100,5586294
2002-12-31,36366773940,18586221,167972254530,5586294
2003-12-31,37707484500,9604990,69110673200,5586294
2004-12-31,47092458420,6104981,46488174470,5586294
2005-12-31,104743012500,28012768,477363346700,5586294
2006-12-31,180437296200,13371594,328730166550,5586294
2007-12-31,299425358400,9259965,571073024250,5586294
2008-12-31,172616484600,6749141,262185509850,5586294


# 종목별 시가총액 

- 예시

In [2]:
from pykrx import stock

df2 = stock.get_market_cap_by_ticker("20201030")
df2 

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,56600,337889692330000,26907844,1541223383000,5969782550
000660,79900,58167388963500,2710477,219064698950,728002365
035420,290000,47636384550000,939833,275981100000,164263395
207940,682000,45124530000000,251134,174660762000,66165000
051910,611000,43131921573000,820454,512035157000,70592343
...,...,...,...,...,...
224020,910,2457000000,0,0,2700000
267810,1450,2243150000,2003,2554350,1547000
238500,443,2074199538,4208,1777904,4682166


In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [9]:
today = datetime.now()
today

datetime.datetime(2023, 8, 14, 17, 18, 23, 728937)

In [11]:
next_month = datetime(today.year, today.month, 1) + relativedelta(months=1)
next_month

datetime.datetime(2023, 9, 1, 0, 0)

In [14]:
this_month_last = next_month + relativedelta(days=-1)
this_month_last.strftime("%Y%m%d")

'20230831'

In [22]:
df = stock.get_market_cap_by_ticker("20220832")
df 

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
060310,0,0,0,0,46271513
000220,0,0,0,0,17232351
000227,0,0,0,0,325560
388720,0,0,0,0,8580846
340930,0,0,0,0,10674744
...,...,...,...,...,...
003000,0,0,0,0,71063049
001270,0,0,0,0,10369886
001275,0,0,0,0,3000000


In [25]:
df['종가'].values[0]

0

In [27]:
df.reset_index()

,티커,종가,시가총액,거래량,거래대금,상장주식수
0,060310,0,0,0,0,46271513
1,000220,0,0,0,0,17232351
2,000227,0,0,0,0,325560
3,388720,0,0,0,0,8580846
4,340930,0,0,0,0,10674744
...,...,...,...,...,...,...
2417,003000,0,0,0,0,71063049
2418,001270,0,0,0,0,10369886
2419,001275,0,0,0,0,3000000
2420,026940,0,0,0,0,20000000


## ETL

In [35]:
from tqdm import tqdm
import easydict
args = easydict.EasyDict()


args.market_cap_path = '../data/market_cap/{year}_{month}.csv'

In [37]:
years = ["200"+str(month) if len(str(month))== 1 else "20"+str(month) for month in range(0,24)]
months = ['0'+str(month) if len(str(month))== 1 else str(month) for month in range(1,13) ]
last_days = ['31', '30', '29', '28', '27', '26', '25']

In [40]:
for year in tqdm(years):
    for month in months:
        for last_day in last_days:
            target_dt = year+month+last_day
            _df = stock.get_market_cap_by_ticker(target_dt)
            if _df['종가'].values[0] == 0:
                continue
            
            print(f'target_dt: {year}-{month}')
            _df = _df.reset_index()
            _df['target_dt'] = target_dt
            _df.rename(columns={
                '티커':'stock_code'
            }, inplace=True)
            save_path = args.market_cap_path.format(year=year, month=month)
            _df.to_csv(save_path, header=True, index=False, encoding='utf-8')
            break # 달마다 한번만...

  0%|          | 0/24 [00:00<?, ?it/s]

target_dt: 2000-01
target_dt: 2000-02
target_dt: 2000-03
target_dt: 2000-04
target_dt: 2000-05
target_dt: 2000-06
target_dt: 2000-07
target_dt: 2000-08
target_dt: 2000-09
target_dt: 2000-10
target_dt: 2000-11


  4%|▍         | 1/24 [00:30<11:40, 30.44s/it]

target_dt: 2000-12
target_dt: 2001-01
target_dt: 2001-02
target_dt: 2001-03
target_dt: 2001-04
target_dt: 2001-05
target_dt: 2001-06
target_dt: 2001-07
target_dt: 2001-08
target_dt: 2001-09
target_dt: 2001-10
target_dt: 2001-11


  8%|▊         | 2/24 [00:59<10:45, 29.35s/it]

...

 92%|█████████▏| 22/24 [13:03<00:57, 28.87s/it]

target_dt: 2021-12
target_dt: 2022-01
target_dt: 2022-02
target_dt: 2022-03
target_dt: 2022-04
target_dt: 2022-05
target_dt: 2022-06
target_dt: 2022-07
target_dt: 2022-08
target_dt: 2022-09
target_dt: 2022-10
target_dt: 2022-11


 96%|█████████▌| 23/24 [13:32<00:28, 28.94s/it]

target_dt: 2022-12
target_dt: 2023-01
target_dt: 2023-02
target_dt: 2023-03
target_dt: 2023-04
target_dt: 2023-05
target_dt: 2023-06
target_dt: 2023-07


100%|██████████| 24/24 [14:27<00:00, 36.15s/it]
